In [1]:
import pandas as pd
from gensim.parsing.preprocessing import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
df_corpus = pd.read_csv('corpus.csv',index_col=0)

In [3]:
corpus = []
for topic in df_corpus:
    corpus += df_corpus[topic].tolist()

answ_corpus = pd.Series(corpus)
answ_corpus = answ_corpus.apply(strip_tags)
answ_corpus = answ_corpus.apply(strip_multiple_whitespaces)
corpus = pd.Series(corpus)
corpus = corpus.apply(lambda x: x.lower())
corpus = corpus.apply(strip_tags)
corpus = corpus.apply(strip_punctuation2)
corpus = corpus.apply(strip_numeric)
corpus = corpus.apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', x))
corpus = corpus.apply(lambda x: re.sub('«|»', ' ', x))
corpus = corpus.apply(strip_multiple_whitespaces)
corpus = corpus.apply(lambda x: x.partition(' – ')[2])

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [5]:
import stop_words
from nltk.stem.snowball import RussianStemmer

corpus_tokens = []
inds_to_drop = []

for i, sentence in en`umerate(corpus[:]):
    tmp_tokens = []
    sp = sentence.split()
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(word)
    if len(tmp_tokens) > 0:
        corpus_tokens.append(tmp_tokens)
    else:
        inds_to_drop.append(i)
#     break
        
print(len(corpus_tokens))#, len(texts))

598


In [6]:
answ_corpus = answ_corpus[~answ_corpus.index.isin(inds_to_drop)]
answ_corpus.index = pd.RangeIndex(len(answ_corpus))

In [7]:
stemmer = RussianStemmer()

In [8]:
corpus_tokens_stem = []

for i, tokens in enumerate((corpus_tokens[:])):
    tmp = [stemmer.stem(word) for word in tokens]
    corpus_tokens_stem.append(tmp)
#     break
    
print(len(corpus_tokens_stem))

598


In [9]:
clear_corpus = []
for token_list in corpus_tokens_stem:
    if len(' '.join(token_list))<2:
        print(token_list)
    clear_corpus.append(' '.join(token_list))
clear_corpus[0]

'кольск проспект район перекрестк улиц шевченк произошл дорожн транспортн происшеств столкнул лад ваз предварительн информац искорежен машин заблокирова девушк пассажир спасател пыта доста автомобил мчс мест работа сотрудник скор помощ'

In [10]:
vectorizer = TfidfVectorizer()
vectorizer.fit(clear_corpus)
X = vectorizer.transform(clear_corpus)

In [11]:
def str_prepr(stringa):
    res_str = stringa.lower()
    res_str = strip_tags(res_str)
    res_str = strip_punctuation2(res_str)
    res_str = strip_numeric(res_str)
    res_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', res_str)
    res_str = re.sub('«|»', ' ', res_str)
    res_str =strip_multiple_whitespaces(res_str)
    sp = res_str.split()
    tmp_tokens = []
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(stemmer.stem(word))
    res_str = ' '.join(tmp_tokens)
    res_embedding = vectorizer.transform([res_str])
#     print(res_str)
    return res_embedding
#     res_str =res_str.partition(' – ')[2]

In [12]:
corpus[0]

'на кольском проспекте в районе перекрёстка на улице шевченко произошло дорожно транспортное происшествие столкнулись лада и ваз по предварительной информации в одной из искорёженных машин заблокирована девушка пассажир спасатели пытаются её достать из автомобиля кроме мчс на месте работают сотрудники скорой помощи и дпс '

In [13]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
nbrs = NearestNeighbors(metric='cosine',radius=1.0,n_neighbors=10).fit(X)
test_str = 'на кольском проспекте в районе перекрёстка на улице шевченко'
distances, indices = nbrs.kneighbors(str_prepr(test_str))

In [14]:
indices

array([[  0, 148,   1,  28,  61,  23,  69,  96, 131, 541]], dtype=int64)

In [15]:
distances

array([[ 0.52344342,  0.75956897,  0.78832967,  0.79640165,  0.82039355,
         0.84198734,  0.86973657,  0.87397946,  0.874021  ,  0.88443941]])

In [16]:
clear_corpus[89]

'утр мончегорск километр трасс кол произошл дорожн транспортн происшеств участник стал легков автомобил ваз грузов автомобил сообщ пресс служб гибдд мурманск област столкновен произошл водител автомобил ваз перекрестк уступ дорог автомобил ваз результат столкновен об машин вылетел встречн полос столкнул грузов автомобил марк втор стык восьмерк вылетел кювет результат дтп пострада человек водител пассажир ваз госпитализирова водител четверк обслуж мест фот александр будник'

In [17]:
nbrs = NearestNeighbors(metric='l2',radius=1.0,n_neighbors=10).fit(X)
test_str = 'на кольском проспекте в районе перекрёстка на улице шевченко произошло дорожно транспортное происшествие столкнулись лада и ваз по предварительной'
distances, indices = nbrs.kneighbors(str_prepr(test_str))

In [18]:
indices

array([[  0,  42,  89,  57,   1, 128,  92,  78,  28,  65]], dtype=int64)

In [19]:
distances

array([[ 0.8212712 ,  1.16738212,  1.2176664 ,  1.23579127,  1.24317771,
         1.24626622,  1.24676911,  1.27044466,  1.27301104,  1.27393093]])

In [20]:
clear_corpus[0]

'кольск проспект район перекрестк улиц шевченк произошл дорожн транспортн происшеств столкнул лад ваз предварительн информац искорежен машин заблокирова девушк пассажир спасател пыта доста автомобил мчс мест работа сотрудник скор помощ'

In [21]:
clear_corpus[89]

'утр мончегорск километр трасс кол произошл дорожн транспортн происшеств участник стал легков автомобил ваз грузов автомобил сообщ пресс служб гибдд мурманск област столкновен произошл водител автомобил ваз перекрестк уступ дорог автомобил ваз результат столкновен об машин вылетел встречн полос столкнул грузов автомобил марк втор стык восьмерк вылетел кювет результат дтп пострада человек водител пассажир ваз госпитализирова водител четверк обслуж мест фот александр будник'

In [22]:
nbrs = NearestNeighbors(metric='l2',radius=10.0,n_neighbors=10).fit(X)
test_str = 'на кольском проспекте в районе перекрёстка на улице шевченко произошло дорожно транспортное происшествие столкнулись лада и ваз по предварительной'
distances, indices = nbrs.kneighbors(str_prepr(test_str))

In [23]:
indices

array([[  0,  42,  89,  57,   1, 128,  92,  78,  28,  65]], dtype=int64)

In [24]:
distances

array([[ 0.8212712 ,  1.16738212,  1.2176664 ,  1.23579127,  1.24317771,
         1.24626622,  1.24676911,  1.27044466,  1.27301104,  1.27393093]])

In [27]:
def search_sys(string):
    preprocessed = str_prepr(string)
    distances, indices = nbrs.kneighbors(str_prepr(string))
    print('по вашему поисковому запросу: "' + string + '" найдено:')
    i = 1
    for text in answ_corpus[indices[0]]:
        print(str(i) + " текст")
        print(text)
        i+=1
        print('')

In [28]:
search_sys('сколько стоит собрать ребенка в школу')

по вашему поисковому запросу: "сколько стоит собрать ребенка в школу" найдено:
1 текст
[ МУРМАНСК/ 3 октября/ БИ-ПОРТ - 28 сентября состоялось праздничное открытие нового учебного года «Школы Росатома». В День атомной промышленности в информационном центре Кольской АЭС около 80 учащихся с 7 по 11 классы стали участниками образовательного проекта. К началу нового учебного года было уже более 300 школьников. Ребята занимаются углублённым изучением технических наук: математики, химии, экологии. Техническое оснащение Атомкласса пополняется современным учебным оборудованием. «Муниципальный проект Школа Росатома предоставляет возможность ребятам нашего города получить качественное образование, быть конкурентоспособными при поступлении в ВУЗы страны. Заинтересованность и личный вклад преподавателей, индивидуальный подход к каждому ученику - залог общего успеха, достойной будущей смены профессионалов для предприятий города и атомной отрасли в целом», – напутствовала собравшихся куратор муницип